We will work today with 'momentum_data.xlsx' file. It contains several tabs:

- The 1st tab contains the momentum factor as an excess return: $\tilde{r}^{\text{mom}}$.

- The 3rd tab contains returns on portfolios corresponding to scored momentum deciles.

    - $r^{\text{mom(1)}}$ denotes the portfolio of stocks in the lowest momentum decile, the 'losers' with the lowest past returns.
    
    - $r^{\text{mom(10)}}$ denotes the portfolio of stocks in the highest momentum decile.
    
- The 4th tab gives portfolios sorted by momentum and size.

    - $r^{\text{momSU}}$ denotes the portfolio of small stocks in the top 3 deciles of momentum scores.
    
    - $r^{\text{momBD}}$ denotes the portfolio of big-stocks in the bottom 3 deciles of momentum scores.

### Task 1.

Check that momentum return, $\tilde{r}^{\text{mom:FF}}$, given in the second tab, is constructed:

$\tilde{r}^{\text{mom:FF}} = (r^{\text{momBU}}+r^{\text{momSU}})/2-
(r^{\text{momBD}}+r^{\text{momSd}})/2$

In [1]:
import pandas as pd
!pip install openpyxl
factors = pd.read_excel('momentum_data.xlsx', 'factors (excess returns)', index_col=0)
momentum = pd.read_excel('momentum_data.xlsx', 'momentum (excess returns)', index_col=0)
size_sorts = pd.read_excel('momentum_data.xlsx', 'size_sorts (total returns)', index_col=0)

In [2]:
momentum['UMD'] == (size_sorts['BIG HiPRIOR']+size_sorts['SMALL HiPRIOR'])/2-(size_sorts['BIG LoPRIOR']+size_sorts['SMALL LoPRIOR'])
r_mom_FF = pd.DataFrame(index=momentum.index)
r_mom_FF['UMD 1']= round(momentum['UMD'],1)
r_mom_FF['UMD 2']= round(((size_sorts['BIG HiPRIOR']+size_sorts['SMALL HiPRIOR'])/2-(size_sorts['BIG LoPRIOR']+size_sorts['SMALL LoPRIOR'])/2),1)
#r_mom_FF
all(r_mom_FF["UMD 1"] == r_mom_FF["UMD 2"])

True

### Task 2

Fill in next table 1 with the appropriate stats for $\tilde{r}^{\text{mom:FF}}$. Please note, that $\tilde{r}^m$ is the excess market return, MKT; $\tilde{r}^v$ is the value portfolio, HML.

<img src='table1.png'>

In [3]:
momentum_perfomance = pd.DataFrame(columns=["mean", "vol", "Sharpe", "skewness", "corr_to_rm", "corr_ro_rv"])
periods = ["1927-2022", "1927-1993", "1994-2008", "2009-2022"]

#створюємо цикл за елементами periods, що, до певного року(з перших до останніх 4ьох символів у periods як зріз), виконує вказаний алгоритм 
for per in periods:
    per_mom = momentum.loc[per[:4]:per[-4:], "UMD"]
    per_fac = factors.loc[per[:4]:per[-4:]]
    #створюємо новий рядок в кінцi з відповідними значеннями
    momentum_perfomance.loc[per] = [per_mom.mean(), per_mom.std(), per_mom.mean()/per_mom.std(), per_mom.skew(), per_mom.corr(per_fac["MKT"]), per_mom.corr(per_fac["HML"])]
momentum_perfomance

,mean,vol,Sharpe,skewness,corr_to_rm,corr_ro_rv
1927-2022,0.006440,0.046953,0.137148,-3.015478,-0.344803,-0.417622
1927-1993,0.007509,0.046185,0.162594,-3.808977,-0.362537,-0.494960
1994-2008,0.008698,0.050677,0.171631,-0.582038,-0.234519,-0.130707
2009-2022,-0.001238,0.045980,-0.026915,-2.811558,-0.377934,-0.385203


### Task 3

Construct your own long-only implementation:

$\tilde{r}^{\text{momU:FF}} = (r^{\text{momBU}}+r^{\text{momSU}})/2 - r^f.$

Note that you have to subtract the risk-free rate (RF) to get the excess return of this portfolio.

Fill out Table 2 for the data in the period 1994-2022.

<img src='table2.png'>

In [4]:
r_mom_FF = pd.DataFrame()
Rf = pd.read_excel("momentum_data.xlsx", 'risk-free rate', index_col=0)

r_mom_FF["Long-and-short (r_mom_FF)"] = momentum["UMD"]
#за формулою знаходимо r_momU_FF
r_mom_FF["Long-only (r_momU_FF)"] = (size_sorts["BIG HiPRIOR"] + size_sorts["SMALL HiPRIOR"])/2 - Rf["RF"]
long_mom = pd.DataFrame(columns=["mean", "vol", "Sharpe", "skewness", "corr_to_rm", "corr_ro_rv"])
long_mom.rename_axis("1994-2022", axis=1)

#проходячисть по колонкам, що ми створили...(де Long-and-short (r_mom_FF) та Long-only (r_momU_FF) колонки)
for per in r_mom_FF.columns:
    per_mom = r_mom_FF.loc["1994":"2022", per]
    per_fac = factors.loc["1994":"2022"]
    #створюємо рядок з відповідними значеннями
    long_mom.loc[per] = [per_mom.mean(), per_mom.std(), per_mom.mean()/per_mom.std(), per_mom.skew(), per_mom.corr(per_fac["MKT"]), per_mom.corr(per_fac["HML"])]
long_mom

,mean,vol,Sharpe,skewness,corr_to_rm,corr_ro_rv
Long-and-short (r_mom_FF),0.003946,0.048672,0.081075,-1.433554,-0.306139,-0.238601
Long-only (r_momU_FF),0.009399,0.051645,0.181984,-0.482872,0.903499,-0.154667


### Task 4

Assess how sensitive the threshold for the 'winners' and 'losers' is in the results. Specifically, we compare three constructions:

- long the top 1 decile and short the bottom 1 deciles:

$\tilde{r}^{\text{momD1}} = r^{\text{mom(10)}}-r^{\text{mom(1)}}$

• long the top 3 deciles and short the bottom 3 deciles:

$\tilde{r}^{\text{momD3}} = 
(r^{\text{mom(8)}}+r^{\text{mom(9)}}+r^{\text{mom(10)}})/3-
(r^{\text{mom(1)}}+r^{\text{mom(2)}}+r^{\text{mom(3)}})/3 = 
\frac 13 \sum^{10}_{k=8} r^{\text{mom(k)}} + 
\frac 13 \sum^{3}_{k=1} r^{\text{mom(k)}}$
 

• long the top 5 deciles and short the bottom 5 deciles:

$\tilde{r}^{\text{momD5}} = 
\frac 15 \sum^{10}_{k=6} r^{\text{mom(k)}} + 
\frac 15 \sum^{5}_{k=1} r^{\text{mom(k)}}$

Compare all three constructions by filling out the stats in the table 3 below for the period 1994-2022.

<img src='table3.png'>

In [5]:
decile = pd.read_excel("momentum_data.xlsx", "deciles (total returns)", index_col=0)
r_mom_D = pd.DataFrame()

#шукаємо значення за формулою(де беремо перші-останні 1,3,5), створюючи генератор за значеннями k
r_mom_D["r_mom_D1"] = decile.iloc[:,9]-decile.iloc[:,0]
r_mom_D["r_mom_D2"] = sum([1/3 *(decile.iloc[:,-1-k] - decile.iloc[:,k]) for k in range(3)])
r_mom_D["r_mom_D5"] = sum([1/5 *(decile.iloc[:,-1-k] - decile.iloc[:,k]) for k in range(5)])
    
    
r_momentum = pd.DataFrame(columns=["mean", "vol", "Sharpe", "skewness", "corr_to_rm", "corr_ro_rv"])
r_momentum.rename_axis("1994-2022", axis=1)
for d in r_mom_D.columns:
    per_mom = r_mom_D.loc["1994":"2022", d]
    per_fac = factors.loc["1994":"2022"]
    r_momentum.loc[d] = [per_mom.mean(), per_mom.std(), per_mom.mean()/per_mom.std(), per_mom.skew(), per_mom.corr(per_fac["MKT"]), per_mom.corr(per_fac["HML"])]
r_momentum

,mean,vol,Sharpe,skewness,corr_to_rm,corr_ro_rv
r_mom_D1,0.006617,0.086011,0.076936,-1.323613,-0.335574,-0.236392
r_mom_D2,0.002850,0.055590,0.051273,-1.355380,-0.365386,-0.229882
r_mom_D5,0.001503,0.038690,0.038837,-1.435409,-0.357936,-0.227501


### Task 5
Use the data provided on both small-stock 'winners', $r^{\text{momSU}}$, and small-stock 'losers', $r^{\text{momSD}}$
to construct a small-stock momentum portfolio,

$
r_t^{\text{momS}} = r^{\text{momSU}} - r^{\text{momSD}}
$

Similarly, use the data provided to construct a big-stock momentum portfolio,

$
r_t^{\text{momB}} = r^{\text{momBU}} - r^{\text{momBD}}
$

Fill out Table 4 over the sample 1994-2022.

<img src='table4.png'>

In [6]:
r_mom_stock = pd.DataFrame()

r_mom_stock["All stocks (r_mom_FF)"] = momentum["UMD"]
r_mom_stock["Small stocks (r_mom_S)"] = size_sorts["SMALL HiPRIOR"] - size_sorts["SMALL LoPRIOR"]
r_mom_stock["Large stocks (r_mom_B)"] = size_sorts["BIG HiPRIOR"] - size_sorts["BIG LoPRIOR"]

r_momentum = pd.DataFrame(columns=["mean", "vol", "Sharpe", "skewness", "corr_to_rm", "corr_ro_rv"])
r_momentum.rename_axis("1994-2022", axis=1)

#класично проходимось колонками зі "stocks"
for st in r_mom_stock.columns:
    per_mom = r_mom_stock.loc["1994":"2022", st]
    per_fac = factors.loc["1994":"2022"]
    r_momentum.loc[st] = [per_mom.mean(), per_mom.std(), per_mom.mean()/per_mom.std(), per_mom.skew(), per_mom.corr(per_fac["MKT"]), per_mom.corr(per_fac["HML"])]
r_momentum

,mean,vol,Sharpe,skewness,corr_to_rm,corr_ro_rv
All stocks (r_mom_FF),0.003946,0.048672,0.081075,-1.433554,-0.306139,-0.238601
Small stocks (r_mom_S),0.005228,0.049541,0.105531,-1.750109,-0.306908,-0.164672
Large stocks (r_mom_B),0.002660,0.052499,0.050667,-0.857615,-0.277984,-0.286919


### Task 6

Re-do Tables 2 and 4 but for the size portfolios of the Value factor. Get this data from Ken French's website, https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html

Specifically, see the following data set:

- Size-sorted portfolios for value. "6 Portfolios Formed on Size and Book-to-Market (2 x 3)"

In [7]:
fact = pd.read_csv("6_Portfolios_2x3.CSV", index_col=0, header=15, nrows=1155, na_values=[-99.99, -999], skip_blank_lines=False)
fact.index = pd.to_datetime(fact.index.astype("str"), format="%Y%m").to_period().to_timestamp('M')

aft = fact.loc["1994":]
aft

,SMALL LoBM,ME1 BM2,SMALL HiBM,BIG LoBM,ME2 BM2,BIG HiBM
1994-01-31,1.8803,2.9469,4.8889,2.4154,4.5770,1.6973
1994-02-28,-0.9196,0.9023,-1.0985,-1.7641,-3.0804,-4.6720
1994-03-31,-6.2311,-4.7222,-4.3361,-4.6933,-3.9696,-3.3803
1994-04-30,-0.8882,0.7042,1.4197,0.6158,1.7767,1.6366
1994-05-31,-2.7048,-1.0815,0.1619,1.4041,1.5369,-0.1128
...,...,...,...,...,...,...
2022-05-31,-4.4206,1.6393,4.1352,-2.6261,3.8821,5.6393
2022-06-30,-4.3163,-6.6654,-11.2053,-7.2567,-8.9137,-12.2993
2022-07-31,13.4602,10.7984,8.7106,11.4389,5.1164,7.9956
2022-08-31,0.2912,-2.2551,-2.2318,-4.7173,-2.0541,-1.5841


### Task 3

Construct your own long-only implementation:

$\tilde{r}^{\text{momU:FF}} = (r^{\text{momBU}}+r^{\text{momSU}})/2 - r^f.$

Note that you have to subtract the risk-free rate (RF) to get the excess return of this portfolio.

Fill out Table 2 for the data in the period 1994-2022.

<img src='table2.png'>

In [8]:
r_mom_FF_n = pd.DataFrame()
Rf = pd.read_excel("momentum_data.xlsx", 'risk-free rate', index_col=0)
momentum_perfomance = pd.DataFrame(columns=["mean", "vol", "Sharpe", "skewness", "corr_to_rm", "corr_ro_rv"])
#за формулою знаходимо r_momU_FF, r_momU_UFF
r_mom_FF_n["Long-and-short (r_mom_FF)"] = (aft["BIG HiBM"] + aft["SMALL HiBM"]) / 2 - (aft["BIG LoBM"] + aft["SMALL LoBM"]) / 2
r_mom_FF_n["Long-only (r_momU_FF)"] = (aft["BIG HiBM"] + aft["SMALL HiBM"]) / 2 - Rf["RF"].loc["1994":]

#проходячисть по колонкам, що ми створили...(де Long-and-short (r_mom_FF) та Long-only (r_momU_FF) колонки)
for per in r_mom_FF_n.columns:
    per_mom = r_mom_FF_n.loc[:, per]
    momentum_perfomance.loc[per] = [per_mom.mean(), per_mom.std(), per_mom.mean()/per_mom.std(), per_mom.skew(), per_mom.corr(per_fac["MKT"]), per_mom.corr(per_fac["HML"])]

momentum_perfomance.rename_axis("1994-2022", axis=1)

1994-2022,mean,vol,Sharpe,skewness,corr_to_rm,corr_ro_rv
Long-and-short (r_mom_FF),0.132581,3.315845,0.039984,0.207751,-0.102250,1.000000
Long-only (r_momU_FF),0.998077,5.495839,0.181606,-0.908702,0.866864,0.330352


### Task 5
Use the data provided on both small-stock 'winners', $r^{\text{momSU}}$, and small-stock 'losers', $r^{\text{momSD}}$
to construct a small-stock momentum portfolio,

$
r_t^{\text{momS}} = r^{\text{momSU}} - r^{\text{momSD}}
$

Similarly, use the data provided to construct a big-stock momentum portfolio,

$
r_t^{\text{momB}} = r^{\text{momBU}} - r^{\text{momBD}}
$

Fill out Table 4 over the sample 1994-2022.

<img src='table4.png'>

In [9]:
r_momentum = pd.DataFrame(columns=["mean", "vol", "Sharpe", "skewness", "corr_to_rm", "corr_ro_rv"])
r_mom_stock_ = pd.DataFrame()
r_mom_stock_["All stocks"] = r_mom_FF_n.iloc[:, 0]
r_mom_stock_["Small stocks"] = aft["SMALL HiBM"] - aft["SMALL LoBM"]
r_mom_stock_["Large stocks"] = aft["BIG HiBM"] - aft["BIG LoBM"]

#класично проходимось колонками зі "stocks"
for st in r_mom_stock_.columns:
    per_mom = r_mom_stock_.loc[:, st]
    r_momentum.loc[st] = [per_mom.mean(), per_mom.std(), per_mom.mean() / per_mom.std(), per_mom.skew(), per_mom.corr(per_fac["MKT"]), per_mom.corr(per_fac["HML"])]
r_momentum.rename_axis("1994-2022", axis=1)

1994-2022,mean,vol,Sharpe,skewness,corr_to_rm,corr_ro_rv
All stocks,0.132581,3.315845,0.039984,0.207751,-0.102250,1.000000
Small stocks,0.306017,3.750472,0.081594,0.271015,-0.269082,0.900306
Large stocks,-0.040855,3.641729,-0.011219,0.015334,0.090916,0.893837


### Task 7

Re-do Table 3 for the decile portfolios of the Value factor. Get this data from Ken French's website, https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html

Specifically, see the following data set:

- "Portfolios Formed on Book-to-Market"

In [10]:
book = pd.read_csv("Portfolios_Formed_on_BE-ME.CSV", index_col=0, header=23, nrows=1155, skip_blank_lines=False,  na_values=[-99.99, -999])
book.index = pd.to_datetime(book.index.astype("str"), format="%Y%m").to_period('M').to_timestamp('M')

new = book.loc["1994":].iloc[:,-10:]
new

,Lo 10,Dec 2,Dec 3,Dec 4,Dec 5,Dec 6,Dec 7,Dec 8,Dec 9,Hi 10
1994-01-31,0.99,3.02,3.73,5.19,4.58,3.38,4.42,0.90,2.76,5.35
1994-02-28,-1.66,-1.62,-1.78,-0.41,-2.78,-2.71,-4.30,-4.32,-4.30,-2.39
1994-03-31,-5.27,-5.11,-3.99,-4.79,-3.59,-4.17,-3.94,-3.56,-2.38,-5.96
1994-04-30,0.65,0.66,0.07,1.16,1.57,2.34,1.63,1.39,1.60,2.13
1994-05-31,0.54,0.23,2.42,0.00,2.62,1.12,0.63,-1.68,1.87,0.43
...,...,...,...,...,...,...,...,...,...,...
2022-05-31,-3.22,-2.59,-1.38,2.46,2.45,3.37,6.99,4.50,6.76,5.94
2022-06-30,-8.01,-6.65,-5.74,-6.57,-8.76,-9.26,-11.19,-11.75,-11.81,-14.49
2022-07-31,14.33,8.09,7.05,5.64,6.07,4.86,6.07,9.27,7.38,6.27
2022-08-31,-5.09,-4.92,-2.99,-3.26,-0.83,-3.75,-0.22,-0.77,-2.82,-1.24


### Task 4

Assess how sensitive the threshold for the 'winners' and 'losers' is in the results. Specifically, we compare three constructions:

- long the top 1 decile and short the bottom 1 deciles:

$\tilde{r}^{\text{momD1}} = r^{\text{mom(10)}}-r^{\text{mom(1)}}$

• long the top 3 deciles and short the bottom 3 deciles:

$\tilde{r}^{\text{momD3}} = 
(r^{\text{mom(8)}}+r^{\text{mom(9)}}+r^{\text{mom(10)}})/3-
(r^{\text{mom(1)}}+r^{\text{mom(2)}}+r^{\text{mom(3)}})/3 = 
\frac 13 \sum^{10}_{k=8} r^{\text{mom(k)}} + 
\frac 13 \sum^{3}_{k=1} r^{\text{mom(k)}}$
 

• long the top 5 deciles and short the bottom 5 deciles:

$\tilde{r}^{\text{momD5}} = 
\frac 15 \sum^{10}_{k=6} r^{\text{mom(k)}} + 
\frac 15 \sum^{5}_{k=1} r^{\text{mom(k)}}$

Compare all three constructions by filling out the stats in the table 3 below for the period 1994-2022.

<img src='table3.png'>

In [11]:
r_mom_D_ = pd.DataFrame()
r_momentum = pd.DataFrame(columns=["mean", "vol", "Sharpe", "skewness", "corr_to_rm", "corr_ro_rv"])
r_momentum

#шукаємо значення за формулою(де беремо перші-останні 1,3,5), створюючи генератор за значеннями k
r_mom_D_["r_mom_D1"] = new.iloc[:,9]-new.iloc[:,0]
r_mom_D_["r_mom_D2"] = sum([1/3 *(new.iloc[:,-1-k] - new.iloc[:,k]) for k in range(3)])
r_mom_D_["r_mom_D5"] = sum([1/5 *(new.iloc[:,-1-k] - new.iloc[:,k]) for k in range(5)])

for d in r_mom_D_.columns:
    per_mom = r_mom_D_.loc["1994":"2022", d]
    r_momentum.loc[d] = [per_mom.mean(), per_mom.std(), per_mom.mean()/per_mom.std(), per_mom.skew(), per_mom.corr(per_fac["MKT"]), per_mom.corr(per_fac["HML"])]
r_momentum.rename_axis("1994-2022", axis=1)

1994-2022,mean,vol,Sharpe,skewness,corr_to_rm,corr_ro_rv
r_mom_D1,0.098754,5.106326,0.019339,-0.152904,0.178362,0.758798
r_mom_D2,0.048029,3.350795,0.014334,-0.289519,0.174831,0.819238
r_mom_D5,0.001467,2.373432,0.000618,-0.039878,0.158168,0.831363
